In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from se3cnn.util.plot import spherical_harmonics_coeff_to_sphere

In [ ]:
import plotly.graph_objects as go

In [ ]:
import vistools
import otp
import cgae.cgae

In [ ]:
def traces_lines(ar, color='red'):
    def trace_two(a):
        x, y, z = a.T
        trace = go.Scatter3d(
            x=x,
            y=y,
            z=z,
            mode='lines',
            line=dict(color=color, width=4)
        )
        return trace
    
    pairs = [np.asarray(i) for i in combinations(ar, 2)]
    traces = [trace_two(i) for i in pairs]
    return traces

# It all starts here

In [ ]:
def load_batched_xyz(pkl):
    args = pkl['args']
    args.device = 'cpu'
    xyz, forces, features = otp.data(pkl['args'])
    n_batches, xyz, forces, features = otp.batch(xyz, forces, features, pkl['args'])
    return n_batches, xyz, forces, features

In [ ]:
ls *.pkl

In [ ]:
# Select save data here
PICKLE = 'edde_short.pkl'

pkl = torch.load(PICKLE, map_location='cpu')
_, xyz, _, features = load_batched_xyz(pkl)

In [ ]:
steps = len(pkl['dynamics'])
epochs = pkl['dynamics'][-1]['epoch']
print(f"There are {steps} steps.")
print(f"Corresponding to {epochs} epochs.")
print(f"i.e. about {steps/epochs} steps/epoch")

In [ ]:
fig, ax = plt.subplots(nrows=2, ncols=1, sharex=True)
loss = [i['loss'] for i in pkl['dynamics']]
loss_ae = [i['loss_ae'] for i in pkl['dynamics']]
loss_fm = [i['loss_fm'] for i in pkl['dynamics']]

ax[0].plot(loss, label='total loss')
ax[0].plot(loss_fm, label='force match')
ax[0].legend()

ax[1].plot(loss_ae, label='autoenc')
ax[1].legend()

fig.tight_layout()
plt.show()
plt.savefig('loss.png')

# Visualize frame

In [ ]:
frame = 45
example = 0

In [ ]:
temp = pkl['summaries'][frame]['temp']
batch = pkl['summaries'][frame]['batch']
geo = xyz[batch, example].detach().numpy()
gumble = pkl['summaries'][frame]['gumble']
st_gumble = pkl['summaries'][frame]['st_gumble']
if len(gumble.shape) == 3:
    gumble = gumble[example]
    st_gumble = st_gumble[example]

cg_xyz = pkl['summaries'][frame]['cg_xyz'][example].detach().numpy()

print(temp, batch)

In [ ]:
plt.imshow(gumble.detach().cpu().numpy().T, aspect=4)
# plt.xticks(np.arange(32))
# plt.yticks(np.arange(3))
# plt.yticks(np.arange(pkl['args'].ncg), ["CG" + str(i+1) for i in range(N_cg)])
plt.show()

In [ ]:
colors = ['red', 'green', 'blue']
colormap = {i: c for i, c in enumerate(colors)}

In [ ]:
# Where are the cg atoms?
data = [vistools.trace_pts(cg_xyz, color='black')]

for color in colors:
    mask = vistools.assignment_to_color(st_gumble, colormap) == color
    data += [vistools.trace_pts(geo[mask.flatten()], color=color)]

fig = go.Figure(data=data)
fig.show()

In [ ]:
# Show reconstruction too.
recon = pkl['summaries'][frame]['reconstructed'][example].detach().numpy()
data = [vistools.trace_pts(cg_xyz, color='black')]

for color in colors:
    mask = vistools.assignment_to_color(st_gumble, colormap) == color
    data += [vistools.trace_pts(recon[mask.flatten()], color=color)]
    data += [vistools.trace_pts(geo[mask.flatten()], color=color, opacity=0.5)]

fig = go.Figure(data=data)
fig.show()